In [21]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision as tv
import torchvision.models as models
from torch.utils.data import DataLoader
import foolbox
from foolbox.models import PyTorchModel

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

num_workers = os.cpu_count()
print(num_workers)

cuda:0
6


In [69]:
root = os.path.join('.', 'dataset_root')
mean, std = [0.13066046], [0.30150425] # based on training set
batch_size = 16
lr = 0.01
momentum=0.9
step_size=10
gamma=0.1
epochs = 20
model_path = 'mnist_cnn.pt'

In [70]:
transform = tv.transforms.Compose([
        tv.transforms.ToTensor(),
        tv.transforms.Normalize(mean, std)])
train_dataset = tv.datasets.MNIST(
    root,
    train=True, 
    download=True,
    transform=transform)
test_dataset = tv.datasets.MNIST(
    root,
    train=False, 
    download=True,
    transform=transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)

In [71]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [72]:
model = Net()
model.load_state_dict(torch.load(model_path))
model.eval()
model = model.to(device)

In [73]:
def validate():
    model.eval()
    total_loss = 0.
    corrects = 0
    
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            batch_size = x.size(0)
            output = model(x)
            loss = F.nll_loss(output, y)
            total_loss += loss.item() * batch_size
            preds = output.max(1, keepdim=True)[1]
            corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(test_loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [74]:
va_loss, va_acc = validate()
print('Test Loss: {:.4f} Accuracy: {:.4f}%'.format(va_loss, va_acc*100.))

Test Loss: 0.0275 Accuracy: 99.1000%


In [75]:
preprocessing = {'mean': mean, 'std': std}
fmodel = PyTorchModel(model, bounds=(0, 1), num_classes=10, preprocessing=preprocessing)

In [76]:
images, labels = next(iter(test_loader))
output = fmodel.forward(images.numpy())
pred = output.argmax(axis=-1)
accuracy = np.sum(pred == labels.numpy()) / len(images)
print(accuracy)

1.0


In [77]:
attack = foolbox.attacks.FGSM(fmodel)
adversarials = attack(images.numpy(), labels.numpy())
print(adversarials.shape)

AssertionError: 

In [78]:
images, labels = foolbox.utils.samples(
    dataset='mnist', 
    batchsize=batch_size, 
    data_format='channels_first', 
    bounds=(0, 1))

In [79]:
attack = foolbox.attacks.FGSM(fmodel)

In [80]:
adversarials = attack(images, labels)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 32 1 3 3, but got 3-dimensional input of size [1, 28, 28] instead